### Vue satellite simple avec couleurs faciles

In [40]:
import folium
import gpxpy
import glob
import hashlib

# Fonction pour générer un hash unique pour chaque trace
def hash_track(points):
    track_str = "".join(f"{lat:.5f},{lon:.5f}" for lat, lon in points)
    return hashlib.md5(track_str.encode()).hexdigest()

# Dossier contenant tes GPX
gpx_files = glob.glob("gpx/*.gpx")  # modifie le chemin si nécessaire

unique_tracks = []
all_points = []

# Lire tous les fichiers et collecter les points, filtrer doublons
for gpx_file in gpx_files:
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                points = [(p.latitude, p.longitude) for p in segment.points]
                if not points:
                    continue
                track_id = hash_track(points)
                if track_id not in unique_tracks:
                    unique_tracks.append(track_id)
                    all_points.extend(points)

# Ajuster le centre et le zoom
center_lat = 46
center_lon = 8
zoom_start = 7.1

# Créer la carte
m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, tiles=None)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Tiles © Esri',
    name="ESRI Satellite",
    overlay=False,
    control=True
).add_to(m)

# Palette sobre pour les tracés
colors = ["darkorange", "crimson", "gold"]

# Ajouter les tracés GPX
idx = 0
for gpx_file in gpx_files:
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                points = [(p.latitude, p.longitude) for p in segment.points]
                if not points:
                    continue
                track_id = hash_track(points)
                if track_id in unique_tracks:
                    folium.PolyLine(
                        points,
                        color=colors[idx % len(colors)],
                        weight=4,
                        opacity=0.9
                    ).add_to(m)
                    unique_tracks.remove(track_id)
                    idx += 1

# Liste complète des villes avec noms locaux
villes = {
    "Paris": (48.8566, 2.3522),
    "Montpellier": (43.6119, 3.8772),
    "Venezia": (45.4408, 12.3155),
    "Marseille": (43.2965, 5.3698),
    "Toulon": (43.1242, 5.9280),
    "Cannes": (43.5528, 7.0174),
    "Nice": (43.7102, 7.2620),
    "Firenze": (43.7696, 11.2558),
    "Bologna": (44.4949, 11.3426),
    "Salzburg": (47.8095, 13.0550),
    "Villach": (46.6167, 13.8500),
    "Genova": (44.4056, 8.9463),
    "Lucca": (43.8429, 10.5027),
    "Udine": (46.0620, 13.2340),
    #"Monaco": (43.7384, 7.4246),
    "München": (48.1351, 11.5820),
    "Roma": (41.9028, 12.4964)
}

# Ajouter un point + nom avec offset en pixels
for nom, coord in villes.items():
    # Petit point
    folium.CircleMarker(
        location=coord,
        radius=4,
        color='white',
        fill=True,
        fill_color='blue',
        fill_opacity=0.9
    ).add_to(m)
    
    # Nom avec offset pixels
    folium.map.Marker(
        location=coord,
        icon=folium.DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f"""<div style="font-size:14px; color:white; font-weight:bold;
                        text-shadow: 1px 1px 2px black;
                        transform: translate(10px, -20px);">{nom}</div>"""
        )
    ).add_to(m)

# Sauvegarder la carte dans un fichier HTML
output_file = "Bike_trip_path_2025.html"
m.save(output_file)
print(f"✅ Carte générée : {output_file}")


✅ Carte générée : Bike_trip_path_2025.html


### Vue satellite simple avec couleurs D+

In [ ]:
import folium
import gpxpy
import glob
import pandas as pd
import matplotlib.colors as colors
import os
import math

# -------------------------------
# 1. Charger D+.txt
# -------------------------------
df = pd.read_csv("D+.txt", sep="\t", header=None)
df = df.rename(columns={0: "GPX", 3: "Difficulte"}) #Difficulte=D+/km
df["GPX"] = df["GPX"].astype(int)

# Palette personnalisée : jaune foncé -> rouge vif
cmap = colors.LinearSegmentedColormap.from_list(
    "yellow_red",
    ["#FFD000", "#FF0000"]  # Jaune foncé -> Rouge vif
)
norm = colors.Normalize(vmin=df["Difficulte"].min(), vmax=df["Difficulte"].max())

def get_color(val):
    rgba = cmap(norm(val))
    return colors.to_hex(rgba)

# -------------------------------
# 2. Créer la carte
# -------------------------------
center_lat = 46
center_lon = 8
zoom_start = 7.1

m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, tiles=None)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Tiles © Esri',
    name="ESRI Satellite",
    overlay=False,
    control=True
).add_to(m)

# -------------------------------
# 3. Tracés GPX colorés par difficulté
# -------------------------------
gpx_files = glob.glob("gpx/*.gpx")

for gpx_file in gpx_files:
    filename = os.path.basename(gpx_file)
    gpx_num = int(os.path.splitext(filename)[0])
    difficulte = df.loc[df["GPX"] == gpx_num, "Difficulte"].values[0]
    color = get_color(difficulte)

    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                points = [(p.latitude, p.longitude) for p in segment.points]
                if points:
                    folium.PolyLine(
                        points,
                        color=color,
                        weight=4,
                        opacity=0.9,
                        tooltip=f"GPX {gpx_num} | Difficulté: {difficulte:.2f}"
                    ).add_to(m)

# -------------------------------
# 4. Villes avec offset léger et ajustement pour proches
# -------------------------------
villes = {
    "Paris": (48.8566, 2.3522),
    "Montpellier": (43.6119, 3.8772),
    "Venezia": (45.4408, 12.3155),
    "Marseille": (43.2965, 5.3698),
    "Toulon": (43.1242, 5.9280),
    "Cannes": (43.5528, 7.0174),
    "Nice": (43.7102, 7.2620),
    "Firenze": (43.7696, 11.2558),
    "Bologna": (44.4949, 11.3426),
    "Salzburg": (47.8095, 13.0550),
    "Villach": (46.6167, 13.8500),
    "Genova": (44.4056, 8.9463),
    "Lucca": (43.8429, 10.5027),
    "Udine": (46.0620, 13.2340),
    "München": (48.1351, 11.5820),
    "Roma": (41.9028, 12.4964),
    "Rapallo": (44.3500, 9.2330)
}

# Paramètres offsets
proximity_thresh = 0.5  # seuil pour considérer 2 villes proches (en degrés)
base_offset = 8          # petit offset léger pour tous
max_offset = 10          # offset maximum pour villes proches
zoom_factor = 1.2        # facteur d'amplification

# Fonction pour calculer distance approximative en degrés
def haversine(lat1, lon1, lat2, lon2):
    return math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

# Offsets initiaux pour toutes les villes
offsets = {nom: (base_offset, -base_offset) for nom in villes.keys()}

# Ajustement pour villes proches
noms = list(villes.keys())
for i in range(len(noms)):
    for j in range(i+1, len(noms)):
        city1, city2 = noms[i], noms[j]
        lat1, lon1 = villes[city1]
        lat2, lon2 = villes[city2]
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist < proximity_thresh:
            offset_val = min(max_offset, (proximity_thresh - dist) / proximity_thresh * max_offset)
            dx = offset_val * zoom_factor
            dy = offset_val * zoom_factor
            offsets[city1] = (-dx, -dy)
            offsets[city2] = (dx, dy)

# Ajouter les villes à la carte
for nom, coord in villes.items():
    folium.CircleMarker(
        location=coord,
        radius=4,
        color='white',
        fill=True,
        fill_color='blue',
        fill_opacity=0.9
    ).add_to(m)

    dx, dy = offsets[nom]
    folium.map.Marker(
        location=coord,
        icon=folium.DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f"""<div style="font-size:14px; color:white; font-weight:bold;
                        text-shadow:1px 1px 2px black;
                        transform: translate({dx}px, {dy}px);">{nom}</div>"""
        )
    ).add_to(m)

# -------------------------------
# 5. Sauvegarder
# -------------------------------
output_file = "Bike_trip_path.html"
m.save(output_file)
print(f"✅ Carte générée : {output_file}")


✅ Carte générée : Bike_trip_path.html


### Avec photos

In [71]:
import folium
import gpxpy
import glob
import pandas as pd
import matplotlib.colors as colors
import os
import math
import shutil

# -------------------------------
# 1. Charger D+.txt
# -------------------------------
df = pd.read_csv("D+.txt", sep="\t", header=None)
df = df.rename(columns={0: "GPX", 3: "Difficulte"})
df["GPX"] = df["GPX"].astype(int)

# Palette personnalisée : jaune foncé -> rouge vif
cmap = colors.LinearSegmentedColormap.from_list(
    "yellow_red",
    ["#FFD000", "#FF0000"]
)
norm = colors.Normalize(vmin=df["Difficulte"].min(), vmax=df["Difficulte"].max())

def get_color(val):
    rgba = cmap(norm(val))
    return colors.to_hex(rgba)

# -------------------------------
# 2. Créer la carte
# -------------------------------
center_lat = 46
center_lon = 8
zoom_start = 7.1

m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, tiles=None)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Tiles © Esri',
    name="ESRI Satellite",
    overlay=False,
    control=True
).add_to(m)

# -------------------------------
# 3. Tracés GPX colorés par difficulté
# -------------------------------
gpx_files = glob.glob("gpx/*.gpx")

for gpx_file in gpx_files:
    filename = os.path.basename(gpx_file)
    gpx_num = int(os.path.splitext(filename)[0])
    difficulte = df.loc[df["GPX"] == gpx_num, "Difficulte"].values[0]
    color = get_color(difficulte)

    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                points = [(p.latitude, p.longitude) for p in segment.points]
                if points:
                    folium.PolyLine(
                        points,
                        color=color,
                        weight=4,
                        opacity=0.9,
                        tooltip=f"GPX {gpx_num} | Difficulté: {difficulte:.2f}"
                    ).add_to(m)

# -------------------------------
# 4. Villes avec noms cliquables et carrousel
# -------------------------------
villes = {
    "Paris": (48.8566, 2.3522),
    "Montpellier": (43.6119, 3.8772),
    "Venezia": (45.4408, 12.3155),
    "Marseille": (43.2965, 5.3698),
    "Toulon": (43.1242, 5.9280),
    "Cannes": (43.5528, 7.0174),
    "Nice": (43.7102, 7.2620),
    "Firenze": (43.7696, 11.2558),
    "Bologna": (44.4949, 11.3426),
    "Salzburg": (47.8095, 13.0550),
    "Villach": (46.6167, 13.8500),
    "Genova": (44.4056, 8.9463),
    "Lucca": (43.8429, 10.5027),
    "Udine": (46.0620, 13.2340),
    "München": (48.1351, 11.5820),
    "Roma": (41.9028, 12.4964),
    "Rapallo": (44.3500, 9.2330)
}

proximity_thresh = 0.5
base_offset = 8
max_offset = 10
zoom_factor = 1.2

def haversine(lat1, lon1, lat2, lon2):
    return math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

# Calcul des offsets pour villes proches
offsets = {nom: (base_offset, -base_offset) for nom in villes.keys()}
noms = list(villes.keys())
for i in range(len(noms)):
    for j in range(i+1, len(noms)):
        city1, city2 = noms[i], noms[j]
        lat1, lon1 = villes[city1]
        lat2, lon2 = villes[city2]
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist < proximity_thresh:
            offset_val = min(max_offset, (proximity_thresh - dist) / proximity_thresh * max_offset)
            dx = offset_val * zoom_factor
            dy = offset_val * zoom_factor
            offsets[city1] = (-dx, -dy)
            offsets[city2] = (dx, dy)

# Créer dossier output_images
os.makedirs("output_images", exist_ok=True)

# Ajouter les villes avec popup carrousel
for nom, coord in villes.items():
    dx, dy = offsets[nom]

    # Préparer carrousel d’images et copier dans output_images
    city_images = glob.glob(f"images/{nom}*.jpg")
    images_html = ""
    for i, img in enumerate(city_images):
        display = "block" if i == 0 else "none"
        basename = os.path.basename(img)
        shutil.copy(img, f"output_images/{basename}")  # Copier l'image
        images_html += f'<div class="mySlides" style="display:{display}; text-align:center;"><img src="output_images/{basename}" width="200px"></div>'

    if images_html:
        html_popup = f"""
        <div style="text-align:center;">
            {images_html}
            <button onclick="plusSlides(-1)">❮</button>
            <button onclick="plusSlides(1)">❯</button>
        </div>
        <script>
        var slideIndex = 1;
        var slides = document.getElementsByClassName('mySlides');
        function showSlides(n) {{
            if(slides.length==0) return;
            slideIndex = n;
            if (n > slides.length) {{slideIndex = 1}}
            if (n < 1) {{slideIndex = slides.length}}
            for (var i = 0; i < slides.length; i++) {{
                slides[i].style.display = "none";
            }}
            slides[slideIndex-1].style.display = "block";
        }}
        function plusSlides(n) {{
            showSlides(slideIndex + n);
        }}
        </script>
        """
    else:
        html_popup = f"<b>{nom}</b><br>Images non disponibles"

    popup = folium.Popup(folium.IFrame(html=html_popup, width=240, height=280), max_width=300)

    # Nom cliquable avec popup
    folium.Marker(
        location=coord,
        popup=popup,
        icon=folium.DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f"""<div style="font-size:14px; color:white; font-weight:bold;
                        text-shadow:1px 1px 2px black;
                        cursor:pointer;
                        transform: translate({dx}px, {dy}px);">{nom}</div>"""
        )
    ).add_to(m)

# -------------------------------
# 5. Sauvegarder
# -------------------------------
output_file = "Bike_trip_path.html"
m.save(output_file)
print(f"✅ Carte générée : {output_file}")


✅ Carte générée : Bike_trip_path.html


### Carrousel base64

In [77]:
import folium
import gpxpy
import glob
import pandas as pd
import matplotlib.colors as colors
import os
import math
import base64

# -------------------------------
# 1. Charger D+.txt
# -------------------------------
df = pd.read_csv("D+.txt", sep="\t", header=None)
df = df.rename(columns={0: "GPX", 3: "Difficulte"})
df["GPX"] = df["GPX"].astype(int)

# Palette jaune -> rouge
cmap = colors.LinearSegmentedColormap.from_list("yellow_red", ["#FFD000", "#FF0000"])
norm = colors.Normalize(vmin=df["Difficulte"].min(), vmax=df["Difficulte"].max())
def get_color(val):
    rgba = cmap(norm(val))
    return colors.to_hex(rgba)

# -------------------------------
# 2. Carte
# -------------------------------
center_lat, center_lon = 46, 8
m = folium.Map(location=[center_lat, center_lon], zoom_start=7.1, tiles=None)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Tiles © Esri',
    name="ESRI Satellite",
    overlay=False,
    control=True
).add_to(m)

# -------------------------------
# 3. Tracés GPX
# -------------------------------
gpx_files = glob.glob("gpx/*.gpx")
for gpx_file in gpx_files:
    filename = os.path.basename(gpx_file)
    gpx_num = int(os.path.splitext(filename)[0])
    difficulte = df.loc[df["GPX"]==gpx_num, "Difficulte"].values[0]
    color = get_color(difficulte)
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                points = [(p.latitude, p.longitude) for p in segment.points]
                if points:
                    folium.PolyLine(points, color=color, weight=4, opacity=0.9,
                                    tooltip=f"GPX {gpx_num} | Difficulté: {difficulte:.2f}").add_to(m)

# -------------------------------
# 4. Villes
# -------------------------------
villes = {
    "Paris": (48.8566, 2.3522),
    "Montpellier": (43.6119, 3.8772),
    "Venezia": (45.4408, 12.3155),
    "Marseille": (43.2965, 5.3698),
    "Toulon": (43.1242, 5.9280),
    "Cannes": (43.5528, 7.0174),
    "Nice": (43.7102, 7.2620),
    "Firenze": (43.7696, 11.2558),
    "Bologna": (44.4949, 11.3426),
    "Salzburg": (47.8095, 13.0550),
    "Villach": (46.6167, 13.8500),
    "Genova": (44.4056, 8.9463),
    "Lucca": (43.8429, 10.5027),
    "Udine": (46.0620, 13.2340),
    "München": (48.1351, 11.5820),
    "Roma": (41.9028, 12.4964),
    "Rapallo": (44.3500, 9.2330)
}

proximity_thresh = 0.5
base_offset, max_offset, zoom_factor = 8, 10, 1.2
def haversine(lat1, lon1, lat2, lon2): return math.sqrt((lat1-lat2)**2 + (lon1-lon2)**2)

# Offsets pour éviter chevauchement
offsets = {nom:(base_offset,-base_offset) for nom in villes.keys()}
noms = list(villes.keys())
for i in range(len(noms)):
    for j in range(i+1, len(noms)):
        c1,c2 = noms[i], noms[j]
        lat1,lon1=villes[c1]; lat2,lon2=villes[c2]
        dist=haversine(lat1,lon1,lat2,lon2)
        if dist<proximity_thresh:
            off=min(max_offset,(proximity_thresh-dist)/proximity_thresh*max_offset)
            dx=dy=off*zoom_factor
            offsets[c1]=(-dx,-dy)
            offsets[c2]=(dx,dy)

# Ajouter villes avec points et popup
for nom, coord in villes.items():
    dx, dy = offsets[nom]

    # Ajouter un petit cercle pour la ville
    folium.CircleMarker(
        location=coord,
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.9
    ).add_to(m)

    # Préparer les images en Base64
    city_images = glob.glob(f"images/{nom}*.jpg")
    images_html = ""
    for i, img in enumerate(city_images):
        display = "block" if i==0 else "none"
        with open(img, "rb") as f:
            data = f.read()
            b64 = base64.b64encode(data).decode()
        images_html += f'<div class="mySlides" style="display:{display}; text-align:center;"><img src="data:image/jpeg;base64,{b64}" width="200px"></div>'

    if images_html:
        html_popup = f"""
        <div style="text-align:center; position:relative;">
            {images_html}
            <button onclick="plusSlides(-1)" style="position:absolute; left:0; top:50%; transform:translateY(-50%); font-size:16px;">❮</button>
            <button onclick="plusSlides(1)" style="position:absolute; right:0; top:50%; transform:translateY(-50%); font-size:16px;">❯</button>
        </div>
        <script>
        var slideIndex = 1;
        var slides = document.getElementsByClassName('mySlides');
        function showSlides(n) {{
            if(slides.length==0) return;
            slideIndex = n;
            if(slideIndex>slides.length) slideIndex=1;
            if(slideIndex<1) slideIndex=slides.length;
            for(var i=0;i<slides.length;i++) slides[i].style.display='none';
            slides[slideIndex-1].style.display='block';
        }}
        function plusSlides(n) {{ showSlides(slideIndex+n); }}
        </script>
        """
    else:
        html_popup = f"<b>{nom}</b><br>Images non disponibles"

    popup = folium.Popup(folium.IFrame(html=html_popup, width=240, height=280), max_width=300)

    # Ajouter un Marker invisible mais cliquable au centre de la ville pour le popup
    folium.Marker(
        location=coord,
        popup=popup,
        icon=folium.DivIcon(icon_size=(0,0))
    ).add_to(m)

    # Ajouter le nom de la ville comme Tooltip au cercle pour info au survol
    folium.CircleMarker(
        location=(coord[0]+dy*0.0005, coord[1]+dx*0.0005),
        radius=0.1, color='transparent', fill_opacity=0,
        tooltip=nom
    ).add_to(m)

# -------------------------------
# 5. Sauvegarder
# -------------------------------
output_file = "Bike_trip_path.html"
m.save(output_file)
print(f"✅ Carte générée : {output_file}")


✅ Carte générée : Bike_trip_path.html


### Perplexity

In [80]:
import folium
import gpxpy
import glob
import pandas as pd
import matplotlib.colors as colors
import os
import math
import base64

# 1. Charger D+.txt
df = pd.read_csv("D+.txt", sep="\t", header=None)
df = df.rename(columns={0: "GPX", 3: "Difficulte"})
df["GPX"] = df["GPX"].astype(int)

# Palette personnalisée : jaune foncé -> rouge vif
cmap = colors.LinearSegmentedColormap.from_list("yellow_red", ["#FFD000", "#FF0000"])
norm = colors.Normalize(vmin=df["Difficulte"].min(), vmax=df["Difficulte"].max())

def get_color(val):
    rgba = cmap(norm(val))
    return colors.to_hex(rgba)

# 2. Créer la carte
center_lat = 46
center_lon = 8
zoom_start = 7.1
m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, tiles=None)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Tiles © Esri', name="ESRI Satellite", overlay=False, control=True
).add_to(m)

# 3. Tracés GPX colorés par difficulté
gpx_files = glob.glob("gpx/*.gpx")
for gpx_file in gpx_files:
    filename = os.path.basename(gpx_file)
    gpx_num = int(os.path.splitext(filename)[0])  # FIXED BUG
    difficulte = df.loc[df["GPX"] == gpx_num, "Difficulte"].values[0]
    color = get_color(difficulte)
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)
    for track in gpx.tracks:
        for segment in track.segments:
            points = [(p.latitude, p.longitude) for p in segment.points]
            if points:
                folium.PolyLine(
                    points,
                    color=color,
                    weight=4,
                    opacity=0.9,
                    tooltip=f"GPX {gpx_num} | Difficulté: {difficulte:.2f}"
                ).add_to(m)

# 4. Villes avec galerie d'images en popup
villes = {
    "Paris": (48.8566, 2.3522),
    "Montpellier": (43.6119, 3.8772),
    "Venezia": (45.4408, 12.3155),
    "Marseille": (43.2965, 5.3698),
    "Toulon": (43.1242, 5.9280),
    "Cannes": (43.5528, 7.0174),
    "Nice": (43.7102, 7.2620),
    "Firenze": (43.7696, 11.2558),
    "Bologna": (44.4949, 11.3426),
    "Salzburg": (47.8095, 13.0550),
    "Villach": (46.6167, 13.8500),
    "Genova": (44.4056, 8.9463),
    "Lucca": (43.8429, 10.5027),
    "Udine": (46.0620, 13.2340),
    "München": (48.1351, 11.5820),
    "Roma": (41.9028, 12.4964),
    "Rapallo": (44.3500, 9.2330)
}
proximity_thresh = 0.5
base_offset = 8
max_offset = 10
zoom_factor = 1.2

def haversine(lat1, lon1, lat2, lon2):
    return math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

offsets = {nom: (base_offset, -base_offset) for nom in villes.keys()}
noms = list(villes.keys())
for i in range(len(noms)):
    for j in range(i+1, len(noms)):
        city1, city2 = noms[i], noms[j]
        lat1, lon1 = villes[city1]
        lat2, lon2 = villes[city2]
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist < proximity_thresh:
            offset_val = min(max_offset, (proximity_thresh - dist) / proximity_thresh * max_offset)
            dx = offset_val * zoom_factor
            dy = offset_val * zoom_factor
            offsets[city1] = (-dx, -dy)
            offsets[city2] = (dx, dy)

photo_folder = "photos" # must contain photos/Paris1.jpg etc.

def build_gallery_html(city, n_images=3):
    gallery_html = f"<h4>{city}</h4><div>"
    for i in range(1, n_images+1):
        img_path = f"{photo_folder}/{city}{i}.jpg"
        if os.path.exists(img_path):
            with open(img_path, "rb") as img_file:
                img_base64 = base64.b64encode(img_file.read()).decode()
            gallery_html += f'<img src="data:image/jpeg;base64,{img_base64}" width="110" style="margin:2px;">'
        else:
            gallery_html += f"<p>Image not found: {img_path}</p>"
    gallery_html += f'<br><a href="{city.lower()}-gallery.html" target="_blank">Voir plus...</a></div>'
    return gallery_html

for nom, coord in villes.items():
    folium.CircleMarker(
        location=coord,
        radius=4,
        color='white',
        fill=True,
        fill_color='blue',
        fill_opacity=0.9
    ).add_to(m)
    dx, dy = offsets[nom]
    html = build_gallery_html(nom)
    iframe = folium.IFrame(html, width=350, height=180)
    popup = folium.Popup(iframe, max_width=350)
    folium.map.Marker(
        location=coord,
        popup=popup,
        icon=folium.DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f"<div style=\"font-size:14px; color:white; font-weight:bold; text-shadow:1px 1px 2px black; transform: translate({dx}px, {dy}px);\">{nom}</div>"
        )
    ).add_to(m)

# 5. Sauvegarder
output_file = "Bike_trip_path.html"
m.save(output_file)
print(f"✅ Carte générée : {output_file}")


✅ Carte générée : Bike_trip_path.html


### Perplexity 2

In [82]:
import folium
import gpxpy
import glob
import pandas as pd
import matplotlib.colors as colors
import os
import math
import base64

# --- 1. Charger D+.txt ---
df = pd.read_csv("D+.txt", sep="\t", header=None)
df = df.rename(columns={0: "GPX", 3: "Difficulte"})
df["GPX"] = df["GPX"].astype(int)

# --- Palette personnalisée jaune->rouge ---
cmap = colors.LinearSegmentedColormap.from_list("yellow_red", ["#FFD000", "#FF0000"])
norm = colors.Normalize(vmin=df["Difficulte"].min(), vmax=df["Difficulte"].max())
def get_color(val):
    rgba = cmap(norm(val))
    return colors.to_hex(rgba)

# --- 2. Créer la carte ---
center_lat = 46
center_lon = 8
zoom_start = 7.1
m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, tiles=None)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Tiles © Esri', name="ESRI Satellite", overlay=False, control=True
).add_to(m)

# --- 3. Tracés GPX colorés par difficulté ---
gpx_files = glob.glob("gpx/*.gpx")
for gpx_file in gpx_files:
    filename = os.path.basename(gpx_file)
    gpx_num = int(os.path.splitext(filename)[0])  # Correction ici
    difficulte = df.loc[df["GPX"] == gpx_num, "Difficulte"].values[0]
    color = get_color(difficulte)
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)
    for track in gpx.tracks:
        for segment in track.segments:
            points = [(p.latitude, p.longitude) for p in segment.points]
            if points:
                folium.PolyLine(
                    points,
                    color=color,
                    weight=4,
                    opacity=0.9,
                    tooltip=f"GPX {gpx_num} | Difficulté: {difficulte:.2f}"
                ).add_to(m)

# --- 4. Villes avec popup galerie d'images ---
villes = {
    "Paris": (48.8566, 2.3522),
    "Montpellier": (43.6119, 3.8772),
    "Venezia": (45.4408, 12.3155),
    "Marseille": (43.2965, 5.3698),
    "Toulon": (43.1242, 5.9280),
    "Cannes": (43.5528, 7.0174),
    "Nice": (43.7102, 7.2620),
    "Firenze": (43.7696, 11.2558),
    "Bologna": (44.4949, 11.3426),
    "Salzburg": (47.8095, 13.0550),
    "Villach": (46.6167, 13.8500),
    "Genova": (44.4056, 8.9463),
    "Lucca": (43.8429, 10.5027),
    "Udine": (46.0620, 13.2340),
    "München": (48.1351, 11.5820),
    "Roma": (41.9028, 12.4964),
    "Rapallo": (44.3500, 9.2330)
}
proximity_thresh = 0.5
base_offset = 8
max_offset = 10
zoom_factor = 1.2

def haversine(lat1, lon1, lat2, lon2):
    return math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

offsets = {nom: (base_offset, -base_offset) for nom in villes.keys()}
noms = list(villes.keys())
for i in range(len(noms)):
    for j in range(i+1, len(noms)):
        city1, city2 = noms[i], noms[j]
        lat1, lon1 = villes[city1]
        lat2, lon2 = villes[city2]
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist < proximity_thresh:
            offset_val = min(max_offset, (proximity_thresh - dist) / proximity_thresh * max_offset)
            dx = offset_val * zoom_factor
            dy = offset_val * zoom_factor
            offsets[city1] = (-dx, -dy)
            offsets[city2] = (dx, dy)

photo_folder = "photos"  # Dossier contenant photos nommées Paris1.jpg etc.

def build_gallery_html(city, n_images=3):
    gallery_html = f"<h4>{city}</h4><div>"
    for i in range(1, n_images+1):
        img_path = f"{photo_folder}/{city}{i}.jpg"
        if os.path.exists(img_path):
            with open(img_path, "rb") as img_file:
                img_base64 = base64.b64encode(img_file.read()).decode()
            gallery_html += f'<img src="data:image/jpeg;base64,{img_base64}" width="110" style="margin:2px;">'
        else:
            gallery_html += f"<p>Image not found: {img_path}</p>"
    gallery_html += f'<br><a href="{city.lower()}-gallery.html" target="_blank">Voir plus...</a></div>'
    return gallery_html

for nom, coord in villes.items():
    folium.CircleMarker(
        location=coord,
        radius=4,
        color='white',
        fill=True,
        fill_color='blue',
        fill_opacity=0.9
    ).add_to(m)
    dx, dy = offsets[nom]
    html = build_gallery_html(nom)
    iframe = folium.IFrame(html, width=370, height=200)
    popup = folium.Popup(iframe, max_width=370)
    folium.Marker(
        location=coord,
        popup=popup,
        icon=folium.DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f"<div style=\"font-size:14px; color:white; font-weight:bold; text-shadow:1px 1px 2px black; transform: translate({dx}px, {dy}px);\">{nom}</div>"
        )
    ).add_to(m)

# --- 5. Sauvegarder ---
output_file = "Bike_trip_path.html"
m.save(output_file)
print(f"✅ Carte générée : {output_file}")


✅ Carte générée : Bike_trip_path.html
